In [1]:
import os
import pandas as pd
import numpy as np
import io
import json
import pickle
import requests
import joblib
import math
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

/home/francisco/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.22) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
def load_data(file):
    df = pd.read_csv(os.path.join("data", file))
    return df

df = load_data("train.csv")

#df_new = load_data("bank_new_observations.csv")
#target='salary'
#df_new.head()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   10000 non-null  int64  
 1   admission_id                 10000 non-null  int64  
 2   patient_id                   10000 non-null  int64  
 3   race                         10000 non-null  object 
 4   gender                       10000 non-null  object 
 5   age                          9724 non-null   object 
 6   weight                       9820 non-null   object 
 7   admission_type_code          9856 non-null   float64
 8   discharge_disposition_code   9930 non-null   float64
 9   admission_source_code        10000 non-null  int64  
 10  time_in_hospital             10000 non-null  int64  
 11  payer_code                   10000 non-null  object 
 12  medical_specialty            10000 non-null  object 
 13  has_prosthesis   

In [7]:
df.describe()

,Unnamed: 0,admission_id,patient_id,admission_type_code,discharge_disposition_code,admission_source_code,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,hemoglobin_level
count,10000.000000,10000.000000,1.000000e+04,9856.000000,9930.000000,10000.000000,10000.000000,9800.000000,10000.000000,9681.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,40697.854000,40697.854000,1.082937e+08,2.015422,3.751762,5.748500,4.340900,42.788776,1.313200,16.032848,0.37870,0.183700,0.623100,7.409000,14.179000
std,23510.882143,23510.882143,7.735092e+07,1.425137,5.318510,4.094524,2.935839,19.666400,1.683327,8.084788,1.32373,0.818304,1.250281,1.922469,1.068013
min,0.000000,0.000000,2.232000e+03,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,10.900000
25%,20367.250000,20367.250000,4.687067e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.00000,0.000000,0.000000,6.000000,13.400000
50%,40715.500000,40715.500000,9.073632e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.00000,0.000000,0.000000,8.000000,14.100000
75%,61071.000000,61071.000000,1.734926e+08,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.00000,0.000000,1.000000,9.000000,15.000000
max,81405.000000,81405.000000,3.766467e+08,8.000000,28.000000,22.000000,14.000000,132.000000,6.000000,70.000000,35.00000,22.000000,18.000000,16.000000,18.100000


In [11]:
df.columns

Index(['Unnamed: 0', 'admission_id', 'patient_id', 'race', 'gender', 'age',
       'weight', 'admission_type_code', 'discharge_disposition_code',
       'admission_source_code', 'time_in_hospital', 'payer_code',
       'medical_specialty', 'has_prosthesis', 'complete_vaccination_status',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'blood_type',
       'hemoglobin_level', 'blood_transfusion', 'max_glu_serum', 'A1Cresult',
       'diuretics', 'insulin', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [12]:
df

,Unnamed: 0,admission_id,patient_id,race,gender,age,weight,admission_type_code,discharge_disposition_code,admission_source_code,...,blood_type,hemoglobin_level,blood_transfusion,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed,readmitted
0,62479,62479,46826964,White,Male,[80-90),?,3.0,1.0,1,...,A-,16.0,False,NONE,None,No,No,No,No,No
1,4543,4543,57647952,Caucasian,Male,[40-50),?,6.0,25.0,17,...,O+,14.2,False,>200,None,No,Yes,No,Yes,No
2,61307,61307,73513008,European,Female,[30-40),?,2.0,1.0,7,...,O+,12.7,False,None,None,No,Yes,No,Yes,No
3,43014,43014,204181182,Caucasian,Female,[40-50),?,1.0,6.0,7,...,O+,12.3,False,None,None,No,Yes,Ch,Yes,Yes
4,64051,64051,47947464,AfricanAmerican,Male,[70-80),?,1.0,11.0,5,...,O+,13.5,False,None,None,No,Yes,Ch,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,55397,55397,125169948,Caucasian,Male,[50-60),?,5.0,1.0,1,...,AB+,15.2,False,NONE,None,No,No,No,No,No
9996,74481,74481,46468368,Black,Female,[60-70),?,1.0,6.0,7,...,A-,13.3,False,None,None,No,Yes,Ch,Yes,No
9997,46182,46182,196276608,AfricanAmerican,Female,[60-70),?,2.0,6.0,4,...,O+,13.7,True,None,None,No,Yes,No,Yes,No
9998,77592,77592,149913090,Caucasian,Male,[50-60),?,1.0,6.0,7,...,B+,14.5,False,None,None,No,No,No,No,No


In [28]:
df_objects = df.select_dtypes(include=['object'])
df_objects

,race,gender,age,weight,payer_code,medical_specialty,complete_vaccination_status,diag_1,diag_2,diag_3,blood_type,max_glu_serum,A1Cresult,diuretics,insulin,change,diabetesMed,readmitted
0,White,Male,[80-90),?,?,?,Complete,185,788,601,A-,NONE,None,No,No,No,No,No
1,Caucasian,Male,[40-50),?,?,?,Complete,786,425,250.01,O+,>200,None,No,Yes,No,Yes,No
2,European,Female,[30-40),?,UN,Family/GeneralPractice,Incomplete,590,250,401,O+,None,None,No,Yes,No,Yes,No
3,Caucasian,Female,[40-50),?,MC,?,Complete,250.82,V85,784,O+,None,None,No,Yes,Ch,Yes,Yes
4,AfricanAmerican,Male,[70-80),?,MC,InternalMedicine,Complete,486,276,276,O+,None,None,No,Yes,Ch,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Caucasian,Male,[50-60),?,HM,?,Complete,574,41,250,AB+,NONE,None,No,No,No,No,No
9996,Black,Female,[60-70),?,MD,InternalMedicine,Complete,197,197,198,A-,None,None,No,Yes,Ch,Yes,No
9997,AfricanAmerican,Female,[60-70),?,?,Cardiology,Complete,414,411,327,O+,None,None,No,Yes,No,Yes,No
9998,Caucasian,Male,[50-60),?,?,?,Complete,531,785,286,B+,None,None,No,No,No,No,No


In [44]:
df.age.unique()

array(['[80-90)', '[40-50)', '[30-40)', '[70-80)', '[90-100)', '[50-60)',
       '[60-70)', '[20-30)', nan, '[10-20)', '[0-10)'], dtype=object)

In [15]:
#age has 276 nan values 

True

In [ ]:
def clean (df):
    df_clean = df.copy()
    df_clean['race'] = df_clean['race'].replace(['AFRICANAMERICAN', 'Afro American'], 'AfricanAmerican')
    df_clean['race'] = df_clean['race'].replace('euro', 'European')
    df_clean['race'] = df_clean['race'].replace(['WHITE', 'White'], 'Caucasian')
    
    df_clean['max_glu_serum'] = df_clean['max_glu_serum'].replace(['NONE'], 'None')
    df_clean['max_glu_serum'] = df_clean['max_glu_serum'].replace(['NORM'], 'Norm')
    
    df_clean['change'] = df_clean['change'].replace(['ch'], 'Yes')
    
    df_clean['weight'] = df_clean['weight'].fillna('?')
    
    df_clean['age'] = df_clean['age'].fillna('?')


In [13]:
df.weight.unique()

array(['?', nan, '[50-75)', '[25-50)', '[75-100)', '[125-150)',
       '[100-125)', '[0-25)', '[150-175)', '[175-200)'], dtype=object)